# Análisis Exploratorio de Datos (EDA) de la tienda Aurelion

TODO:
* Preprocesamiento de datos:
    * Revisión de datos, dimensionalidad, etc.
    * Estadísticas descriptivas básicas.
    * Tratamiento de valores faltantes.
    * Corrección de tipos de datos.
    * Estandarización de formatos y normalizaciones.
    * Eliminación de duplicados.
    * Tratamiento de inconsistencias.
    * Detección y tratamiento de Outliers.
* Exploración de datos:
    * Análisis univariable (distribución y comportamiento de cada variable).
    * Análisis bivariante (asociaciones y correlaciones).
    * Análisis de tendencias.
* Conclusiones.

## Preprocesamiento de datos

### Importación de librerías y carga de datos

In [55]:
import pandas as pd
import numpy as np

ventas = pd.read_excel("./data/ventas.xlsx")
productos = pd.read_excel("./data/productos_corregidos.xlsx")
clientes = pd.read_excel("./data/clientes.xlsx")
detalle_ventas = pd.read_excel("./data/detalle_ventas.xlsx")

### Inspección inicial de los datos

In [56]:
print(detalle_ventas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_venta         343 non-null    int64 
 1   id_producto      343 non-null    int64 
 2   nombre_producto  343 non-null    object
 3   cantidad         343 non-null    int64 
 4   precio_unitario  343 non-null    int64 
 5   importe          343 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 16.2+ KB
None


In [57]:
print(ventas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_venta        120 non-null    int64         
 1   fecha           120 non-null    datetime64[ns]
 2   id_cliente      120 non-null    int64         
 3   nombre_cliente  120 non-null    object        
 4   email           120 non-null    object        
 5   medio_pago      120 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 5.8+ KB
None


In [53]:
print(productos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id_producto          100 non-null    int64 
 1   nombre_producto      100 non-null    object
 2   categoria            100 non-null    object
 3   precio_unitario      100 non-null    int64 
 4   categoria_corregida  100 non-null    object
dtypes: int64(2), object(3)
memory usage: 4.0+ KB
None


In [54]:
print(clientes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_cliente      100 non-null    int64         
 1   nombre_cliente  100 non-null    object        
 2   email           100 non-null    object        
 3   ciudad          100 non-null    object        
 4   fecha_alta      100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 4.0+ KB
None


Dado lo anterior, concluimos que es posible la unificación de los 4 dataframes:
* ventas y detalle_ventas en id_venta.
* productos y detalle_ventas en id_producto.
* clientes y ventas en id_cliente.
A continuación se muestra la unificación con base en nuestras observaciones:

In [69]:
df_unified = ventas.merge(detalle_ventas, on="id_venta", suffixes=("_ventas", "_dventas"))
df_unified = df_unified.merge(productos, on="id_producto", suffixes=("_dventas", "_producto"))
df_unified = df_unified.merge(clientes, on="id_cliente", suffixes=("_ventas", "_cliente"))

df_unified.head()

,id_venta,fecha,id_cliente,nombre_cliente_ventas,email_ventas,medio_pago,id_producto,nombre_producto_dventas,cantidad,precio_unitario_dventas,importe,nombre_producto_producto,categoria,precio_unitario_producto,categoria_corregida,nombre_cliente_cliente,email_cliente,ciudad,fecha_alta
0,1,2024-06-19,62,Guadalupe Romero,guadalupe.romero@mail.com,tarjeta,90,Toallas Húmedas x50,1,2902,2902,Toallas Húmedas x50,Limpieza,2902,Limpieza,Guadalupe Romero,guadalupe.romero@mail.com,Carlos Paz,2023-03-03
1,2,2024-03-17,49,Olivia Gomez,olivia.gomez@mail.com,qr,82,Aceitunas Negras 200g,5,2394,11970,Aceitunas Negras 200g,Limpieza,2394,Alimentos,Olivia Gomez,olivia.gomez@mail.com,Rio Cuarto,2023-02-18
2,2,2024-03-17,49,Olivia Gomez,olivia.gomez@mail.com,qr,39,Helado Vainilla 1L,5,469,2345,Helado Vainilla 1L,Alimentos,469,Alimentos,Olivia Gomez,olivia.gomez@mail.com,Rio Cuarto,2023-02-18
3,2,2024-03-17,49,Olivia Gomez,olivia.gomez@mail.com,qr,70,Fernet 750ml,2,4061,8122,Fernet 750ml,Limpieza,4061,Alimentos,Olivia Gomez,olivia.gomez@mail.com,Rio Cuarto,2023-02-18
4,2,2024-03-17,49,Olivia Gomez,olivia.gomez@mail.com,qr,22,Medialunas de Manteca,1,2069,2069,Medialunas de Manteca,Limpieza,2069,Alimentos,Olivia Gomez,olivia.gomez@mail.com,Rio Cuarto,2023-02-18


Podemos observar más a detalle los nombres de las columnas para eliminar aquellas que estén duplicadas.

In [83]:
df_unified.info()
print(df_unified.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_venta                  343 non-null    int64         
 1   fecha                     343 non-null    datetime64[ns]
 2   id_cliente                343 non-null    int64         
 3   nombre_cliente_ventas     343 non-null    object        
 4   email_ventas              343 non-null    object        
 5   medio_pago                343 non-null    object        
 6   id_producto               343 non-null    int64         
 7   nombre_producto_dventas   343 non-null    object        
 8   cantidad                  343 non-null    int64         
 9   precio_unitario_dventas   343 non-null    int64         
 10  importe                   343 non-null    int64         
 11  nombre_producto_producto  343 non-null    object        
 12  categoria             

### Estadísticas descriptivas básicas

In [74]:
df_unified.describe(include='all')

,id_venta,fecha,id_cliente,nombre_cliente_ventas,email_ventas,medio_pago,id_producto,nombre_producto_dventas,cantidad,precio_unitario_dventas,importe,nombre_producto_producto,categoria,precio_unitario_producto,categoria_corregida,nombre_cliente_cliente,email_cliente,ciudad,fecha_alta
count,343.000000,343,343.000000,343,343,343,343.000000,343,343.000000,343.000000,343.000000,343,343,343.000000,343,343,343,343,343
unique,NaN,NaN,NaN,64,67,4,NaN,95,NaN,NaN,NaN,95,2,NaN,2,64,67,6,NaN
top,NaN,NaN,NaN,Agustina Flores,agustina.flores@mail.com,efectivo,NaN,Queso Rallado 150g,NaN,NaN,NaN,Queso Rallado 150g,Limpieza,NaN,Alimentos,Agustina Flores,agustina.flores@mail.com,Rio Cuarto,NaN
freq,NaN,NaN,NaN,15,15,111,NaN,9,NaN,NaN,NaN,9,178,NaN,286,15,15,104,NaN
mean,61.492711,2024-03-30 14:12:14.693877504,45.539359,NaN,NaN,NaN,49.139942,NaN,2.962099,2654.495627,7730.078717,NaN,NaN,2654.495627,NaN,NaN,NaN,NaN,2023-02-14 12:56:40.583090432
min,1.000000,2024-01-02 00:00:00,1.000000,NaN,NaN,NaN,1.000000,NaN,1.000000,272.000000,272.000000,NaN,NaN,272.000000,NaN,NaN,NaN,NaN,2023-01-01 00:00:00
25%,31.000000,2024-02-10 12:00:00,20.000000,NaN,NaN,NaN,23.000000,NaN,2.000000,1618.500000,3489.000000,NaN,NaN,1618.500000,NaN,NaN,NaN,NaN,2023-01-20 00:00:00
50%,61.000000,2024-03-26 00:00:00,43.000000,NaN,NaN,NaN,47.000000,NaN,3.000000,2512.000000,6702.000000,NaN,NaN,2512.000000,NaN,NaN,NaN,NaN,2023-02-12 00:00:00
75%,93.000000,2024-05-21 00:00:00,67.000000,NaN,NaN,NaN,76.000000,NaN,4.000000,3876.000000,10231.500000,NaN,NaN,3876.000000,NaN,NaN,NaN,NaN,2023-03-08 00:00:00
max,120.000000,2024-06-28 00:00:00,100.000000,NaN,NaN,NaN,100.000000,NaN,5.000000,4982.000000,24865.000000,NaN,NaN,4982.000000,NaN,NaN,NaN,NaN,2023-04-10 00:00:00
